In [1]:
from web3 import Web3, HTTPProvider
web3 = Web3(HTTPProvider("HTTP://127.0.0.1:7545"))

In [2]:
list = web3.eth.accounts

In [3]:
from solc import compile_standard

In [4]:
# read sol file. 
with open("receiveEther.sol") as f:
    data = f.read()

In [5]:
input = {
    "language": "Solidity",
    "sources": {
        "receiveEther.sol": {
            "content": data
        }
    },
    "settings":
    {
        "outputSelection": {
            "*": {
                "*": [
                    "metadata", "evm.bytecode", "evm.bytecode.sourceMap"
                ]
            }
        }
    }
}

In [6]:
compiledSol = compile_standard(input) 

In [7]:
bytecode = compiledSol['contracts']['receiveEther.sol']['receiveEther']['evm']['bytecode']['object']

In [8]:
import json
abi = json.loads(compiledSol['contracts']['receiveEther.sol']['receiveEther']['metadata'])['output']['abi']

In [9]:
CalcContract = web3.eth.contract(abi=abi, bytecode= bytecode)

In [10]:
tx_hash = CalcContract.constructor().transact({
    "from":list[0]
})

In [11]:
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash) 

In [12]:
# The contract contains payable fallback function.
# the result is payable address. 
conAddr = tx_receipt.contractAddress

### send ether to contract address. 

In [13]:
web3.eth.sendTransaction({
    'from' :list[0],
    'to' : conAddr,
    'value': Web3.toWei(0.15, 'ether')
})

HexBytes('0xcab66966ae46720ec95766a76f20a3c7a749e25abf98d1a9d8283c06266cfd24')

In [14]:
Instance = web3.eth.contract(
    address=conAddr,
    abi = abi
)

In [17]:
# interact with the SC 
Instance.functions.getnumber().call()

150000000000000000

In [18]:
web3.eth.getBalance(conAddr)

150000000000000000